In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt # function in matplotlib that produces quick and easy plots
import cartopy.crs as ccrs # for projection
import cartopy.feature as cfeature # for map features 
from cartopy.util import add_cyclic_point


ModuleNotFoundError: No module named 'cartopy'

## (1) Writing a simple function (including some additional practice with flow control)
### Write a simple function, such that
- You pass a numpy array of four numbers into it – chilly winter temperatures of 20.4, 21.6, 19.8, 22.2 (degrees F, but you don't have to worry about units)
<br><br>
- It will return the maximum value of those numbers
  - Yes, we have actual built-in functions for this - but this is good practice
<br><br>
- You cannot use any pre-existing functions in doing this (except range, arange, etc)
<br><br>
- Do not hardwire anything in this function - this function should also work if you pass in an array of a different size than that of the winter temperatures in this example
<br><br>
- Actually, prove this to me by creating an array of FIVE winter temperatures of your choice and showing me that your function still works!
<br><br>
- For this Q, you will use logical/comparison operations, as well as flow control structures

In [8]:
# Write a simple function such that it takes a numpy array of size m and returns the maximum value in the array, without using any pre-existing functions except range(), arange() 
def max_value(arr):
    max_val = 0
    for i in range(len(arr)):
        if arr[i] > max_val:
            max_val = arr[i]
    return max_val

In [9]:
# temp1 = np.array([20.4, 21.6, 19.8, 22.2])
temp1 = np.array([20.4, 21.6, 19.8, 22.2])
print(max_value(temp1))

22.2


In [ ]:
# create an array of five winter temperatures
temp2 = np.array([20.4, 21.6, 19.8, 22.2, 21.2])
print(max_value(temp2))

## (2) Practice with some spatial data using basic Xarray!
- You were just briefly acquainted with the Xarray package this week
<br><br>
- We'll do much more in Xarray in a few weeks, but for now I just want you to start thinking about, and working a bit with, spatial data using this package
<br><br>
- In this Q, you'll read in oceanography data - some metadata here: https://www.metoffice.gov.uk/hadobs/en4/en4-2-2-analysis-file-format.html
<br><br> 
- And evaluate a few basic characteristics of the structure of it, access elements, and then I'll create a plot for you

#### Read in 'EN.4.2.2.f.analysis.g10.202001.nc' - this is a DATASET as there are multiple data variables in it - temperature, salinity, measures of uncertainty for both, etc.

In [2]:
data = xr.open_dataset('EN.4.2.2.f.analysis.g10.202001.nc')

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

#### (a.)  Print the data variables in this netcdf file, using the specific function (see your Intro to Xarray lesson)

#### (b.)  Print the coordinates in this netcdf file

In [3]:
data.coords

Coordinates:
  * depth    (depth) float32 5.022 15.08 25.16 ... 4.752e+03 5.051e+03 5.35e+03
  * lat      (lat) float32 -83.0 -82.0 -81.0 -80.0 -79.0 ... 86.0 87.0 88.0 89.0
  * lon      (lon) float32 1.0 2.0 3.0 4.0 5.0 ... 356.0 357.0 358.0 359.0 360.0
  * time     (time) datetime64[ns] 2020-01-16T12:00:00

#### (c.)   Let's focus just on temperature. 

- (i.) How many dimensions does temperature have?
<br><br>
- (ii.) What are these dimensions, and in what order are they in this variable's structure?
<br><br>
- (iii.) Access just the temperature data for the first and only time, and the first depth (aka, the vertical level of 5 m below the surface), but all latitudes and longitudes and save it to a new variable called 'temperature'.  
   - Do this using NumPy array indexing!  
   - Later, we'll use special functionality for indexing in Xarray that will allow us to no longer need to keep track of dimension order :)
<br><br>

**NOTES**
- Keep in mind: if a dimension is of length ONE, that would technically not count towards your total # of dimensions. BUT, when you index this variable in (iii.), you'll use '0' to access the one and only value of this 'dimension of length 1', since it is still part of the temperature data structure

In [4]:
# (iii.) Access just the temperature data for the first and only time, and the first depth (aka, the vertical level of 5 m below the surface), but all latitudes and longitudes and save it to a new variable called 'temperature'.  
   # -  Do this using NumPy array indexing!  
   # - Later, we'll use special functionality for indexing in Xarray that will allow us to no longer need to keep track of dimension order :)

temperature = data['temperature'][0, 0, :, :]
print(temperature.shape)


In [ ]:
# get masked array of temperature data
np.ma.masked_less(temperature, 0)

#### (d.)  I create your contour plot of Jan 2020 average temperature at depth = 5 m on a map for you. 
#### YOUR TASK: CAREFULLY REVIEW EACH STEP OF THIS AND ASK IF ANYTHING IS UNCLEAR AS YOU'LL BE DOING SOMETHING VERY SIMILAR ON YOUR WEEK 5 ASSIGNMENT!  PLEASE ALSO CAREFULLY WATCH YOUR 2D PLOTTING, PART II LESSON, OR RE-WATCH. MAPPING IS IMPORTANT STUFF!

In [5]:
proj=ccrs.PlateCarree() # this is a very common projection, but not the only one!

#Create a figure with an axes object on which we will plot. Pass the projection to that axes
fig, ax = plt.subplots(figsize=(15,15),subplot_kw=dict(projection=proj))

# Grab our lat and lon coordinate variables from the Netcdf file
lon = data['lon']
lat = data['lat'] 

# Add cyclic point since there's a data gap in this data set at the Prime Meridian
temperature2, lon2 = add_cyclic_point(temperature, coord=lon)
# use a new variable for our now-modified temperature and longitude variables b/c if we don't and re-run just this cell
# multiple times, you'll keep adding a longitude point (oops!)

# Create our grid
# contourf needs this so it understands the grid it is plotting on
X, Y = np.meshgrid(lon2, lat) # using our slightly modified longitude

# Create our contour plot!
cs = ax.contourf(X,Y,temperature2,transform=proj) # good practice to also specify the transform, as otherwise
                                                  # Cartopy assumes your transform is whatever your projection is
                                                  # this may not always be true
                                                  # more on this later

# Create our colorbar
cb=fig.colorbar(cs,shrink=0.4) # shrink option does what it sounds like to the colorbar!
cb.set_label("Temperature (K)") # set the colorbar label
# The temperature is in Kelvin

# Some options involving the gridlines
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Add coastlines!
ax.coastlines()

# Add a title!
ax.set_title('Ocean temperature at 5 m below the surface, Jan 2020')

Text(0.5, 1.0, 'Ocean temperature at 5 m below the surface, Jan 2020')